In [7]:
import polars as pl

from google.cloud import bigquery
from google.oauth2 import service_account

pl.Config.set_fmt_str_lengths(200)
pl.Config.set_fmt_float("full")

polars.config.Config

# ChatGPT Instructions

To authenticate with Google Cloud services like BigQuery in Python without using a browser-based authentication flow, you can use service account credentials. A service account is a special kind of account used by applications (rather than users) to interact with Google Cloud services. Here’s how you can set it up:

1. Create a Service Account and Download the JSON Key File
Go to the Google Cloud Console (console.cloud.google.com).
Select your project.
Navigate to IAM & Admin > Service Accounts.
Click Create Service Account, give it a name, describe its role, and grant it the necessary roles (like BigQuery Admin for full access to BigQuery).
After creating the service account, click on it to manage keys.
Add a new key and select JSON type. This will download the JSON key file to your computer.
2. Authenticate in Python Using the Service Account
With the JSON key file, you can authenticate your application in Python as follows:

python
Copy code
from google.cloud import bigquery
from google.oauth2 import service_account

# Path to your service account key file
service_account_key_path = 'path/to/your/service-account-file.json'

# Create credentials using the service account key file
credentials = service_account.Credentials.from_service_account_file(
    service_account_key_path
)

# Use the credentials to create a client for your project
client = bigquery.Client(credentials=credentials, project=credentials.project_id)

# Now you can use this client to perform operations with BigQuery
3. Environment Variable (Alternative Method)
Alternatively, you can set an environment variable that the Google client libraries use to automatically locate the credentials file. This is often more convenient as it reduces the need to modify your code to handle authentication.

In your system’s shell (e.g., Bash), you can set the environment variable like this:

bash
Copy code
export GOOGLE_APPLICATION_CREDENTIALS="/path/to/your/service-account-file.json"
In Python, you can set this environment variable within your script (though setting it in the system's shell is recommended for security and modularity):

python
Copy code
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/path/to/your/service-account-file.json"

# BigQuery 

In [2]:
# Create credentials using the service account key file
credentials = service_account.Credentials.from_service_account_file(
    "../eden-relay-8817ef6d9a58.json"
)

# Use the credentials to create a client for your project
client = bigquery.Client(credentials=credentials,
                         project=credentials.project_id)

In [ ]:
# TODO - add xatu data query for beacon chain payload data...

In [19]:
sql = """
    WITH RankedBids AS (
        SELECT 
            relay,
            slot,
            block_hash,
            timestamp,
            block_timestamp,
            value,
            -- RANK() OVER(PARTITION BY slot, block_hash ORDER BY timestamp ASC) AS rank_asc,
            RANK() OVER(PARTITION BY slot, block_hash ORDER BY timestamp DESC) AS rank_desc
        FROM `eden-data-public.mev_boost.bids` 
        WHERE TIMESTAMP_TRUNC(timestamp, DAY) > TIMESTAMP("2024-04-15")
    ),
    AggregatedBids AS (
        SELECT
            slot,
            block_hash,
            relay,
            block_timestamp,
            -- MIN(timestamp) AS earliest_bid_timestamp,
            MAX(timestamp) AS latest_bid_timestamp,
            -- MAX(CASE WHEN rank_asc = 1 THEN value END) / POW(10, 18) AS earliest_bid_value,
            MAX(CASE WHEN rank_desc = 1 THEN value END) / POW(10, 18) AS latest_bid_value
        FROM RankedBids
        GROUP BY slot, block_hash, relay, block_timestamp
    )
    SELECT * FROM AggregatedBids
    """

In [20]:
query_df = client.query(sql).to_dataframe()

In [21]:
pl_df = pl.DataFrame(query_df)

In [2]:
pl_df = pl.scan_parquet("agg_bids_cache.parquet")
# pl_df.write_parquet("agg_bids_cache.parquet")

In [3]:
pl_df.select(pl.len()).collect()

len
u32
55496166


In [40]:
relays_latest_bids_rec_df = (
    pl_df.sort(by="slot")
    .with_columns(
        (pl.col("latest_bid_timestamp") - pl.col("block_timestamp")).alias(
            "bid_since_block_timestamp"
        ),
    )
    .group_by("slot", "relay")
    .agg(
        pl.col("bid_since_block_timestamp")
        .max()
        .alias("latest_relay_bid_since_block_timestamp"),
        pl.col("latest_bid_value").max().alias("max_latest_relay_bid_value"),
    )
).collect(streaming=True)

In [50]:
relays_latest_bids_rec_df.select("relay").unique()

relay
str
"""bloxrouteMaxProfit"""
"""agnostic"""
"""ultrasound"""
"""eden"""
"""flashbots"""
"""manifold"""
"""aestus"""


In [80]:
bloxroute_table = (
    relays_latest_bids_rec_df
    # .filter(pl.col("relay") == "bloxrouteMaxProfit")
    .with_columns(
        pl.col("latest_relay_bid_since_block_timestamp")
        .cast(pl.Duration(time_unit="ms"))
        .alias("latest_relay_bid_since_block_timestamp")
    )
    .with_columns(
        (pl.col("latest_relay_bid_since_block_timestamp") / 1000)
        .cast(pl.Float64)
        .alias("latest_bid_since_block_seconds")
    )
    .sort(by="latest_relay_bid_since_block_timestamp", descending=True)
    .filter(pl.col("latest_bid_since_block_seconds") < 7)
    .filter(pl.col("latest_bid_since_block_seconds") > 0)
)

In [81]:
bloxroute_table.plot.kde(y='latest_bid_since_block_seconds', by='relay', xlabel='block bid time (seconds)', title='latest block bids by relay')

:NdOverlay   [relay]
   :Distribution   [latest_bid_since_block_seconds]   (Density)